In [1]:
from collections import defaultdict
from itertools import chain, combinations
# Data Transaksi
transactions = [
    ['Roti', 'Susu'],
    ['Roti', 'Mentega', 'Telur'],
    ['Mentega', 'Susu', 'Keju'],
    ['Roti', 'Mentega', 'Susu'],
    ['Roti', 'Mentega', 'Susu', 'Keju']
]

min_support = 3

In [2]:
def get_item_frequency(transactions):
    item_frequency = defaultdict(int)
    for transaction in transactions:
        for item in transaction:
            item_frequency[item] += 1
    return item_frequency

item_frequency = get_item_frequency(transactions)
print("Item Frequency:", item_frequency)

Item Frequency: defaultdict(<class 'int'>, {'Roti': 4, 'Susu': 4, 'Mentega': 4, 'Telur': 1, 'Keju': 2})


In [3]:
def create_header_table(item_frequency, min_support):
    header_table = {}
    for item, count in item_frequency.items():
        if count >= min_support:
            header_table[item] = [count, None]
    return header_table

header_table = create_header_table(item_frequency, min_support)
print("Header Table:", header_table)


Header Table: {'Roti': [4, None], 'Susu': [4, None], 'Mentega': [4, None]}


In [4]:
class TreeNode:
    def __init__(self, item, count, parent):
        self.item = item
        self.count = count
        self.parent = parent
        self.children = {}
        self.link = None

    def increment(self, count):
        self.count += count

def update_header_table(node, target_node):
    while node.link is not None:
        node = node.link
    node.link = target_node

def update_tree(items, node, header_table):
    if items:
        first_item = items[0]
        if first_item in node.children:
            node.children[first_item].increment(1)
        else:
            new_node = TreeNode(first_item, 1, node)
            node.children[first_item] = new_node
            if header_table[first_item][1] is None:
                header_table[first_item][1] = new_node
            else:
                update_header_table(header_table[first_item][1], new_node)
        update_tree(items[1:], node.children[first_item], header_table)

def create_fp_tree(transactions, min_support):
    header_table = create_header_table(get_item_frequency(transactions), min_support)
    root = TreeNode('Null', 1, None)
    for transaction in transactions:
        filtered_transaction = [item for item in transaction if item in header_table]
        filtered_transaction.sort(key=lambda item: header_table[item][0], reverse=True)
        update_tree(filtered_transaction, root, header_table)
    return root, header_table

fp_tree, header_table = create_fp_tree(transactions, min_support)

def print_tree(node, indent=0):
    print('  ' * indent, node.item, ' ', node.count)
    for child in node.children.values():
        print_tree(child, indent + 1)

print_tree(fp_tree)

 Null   1
   Roti   4
     Susu   1
     Mentega   3
       Susu   2
   Mentega   1
     Susu   1


In [5]:
def ascend_tree(node):
    prefix_path = []
    while node.parent is not None:
        prefix_path.append(node.item)
        node = node.parent
    return prefix_path[1:]

def find_prefix_paths(base_pattern, node):
    conditional_patterns = []
    while node is not None:
        prefix_path = ascend_tree(node)
        if len(prefix_path) > 0:
            conditional_patterns.append((prefix_path, node.count))
        node = node.link
    return conditional_patterns

def mine_tree(header_table, min_support, prefix, frequent_itemsets):
    sorted_items = [item for item, nodes in sorted(header_table.items(), key=lambda p: p[1][0])]
    for base_pattern in sorted_items:
        new_frequent_set = prefix.copy()
        new_frequent_set.add(base_pattern)
        frequent_itemsets.append(new_frequent_set)
        conditional_pattern_base = find_prefix_paths(base_pattern, header_table[base_pattern][1])
        print(f"Conditional pattern base for {base_pattern}: {conditional_pattern_base}")
        conditional_transactions = []
        for pattern, count in conditional_pattern_base:
            for _ in range(count):
                conditional_transactions.append(pattern)
        if conditional_transactions:
            conditional_tree, new_header_table = create_fp_tree(conditional_transactions, min_support)
            if new_header_table is not None:
                mine_tree(new_header_table, min_support, new_frequent_set, frequent_itemsets)

frequent_itemsets = []
mine_tree(header_table, min_support, set(), frequent_itemsets)

# Tampilkan hasil
print("Frequent Itemsets:")
for itemset in frequent_itemsets:
    print(itemset)

Conditional pattern base for Roti: []
Conditional pattern base for Susu: [(['Roti'], 1), (['Mentega'], 1), (['Mentega', 'Roti'], 2)]
Conditional pattern base for Roti: [(['Mentega'], 2)]
Conditional pattern base for Mentega: []
Conditional pattern base for Mentega: [(['Roti'], 3)]
Conditional pattern base for Roti: []
Frequent Itemsets:
{'Roti'}
{'Susu'}
{'Susu', 'Roti'}
{'Susu', 'Mentega'}
{'Mentega'}
{'Mentega', 'Roti'}


In [6]:
# Menghitung support untuk setiap itemset
itemset_support = defaultdict(int)
for itemset in frequent_itemsets:
    for transaction in transactions:
        if itemset.issubset(transaction):
            itemset_support[frozenset(itemset)] += 1

# Menampilkan support untuk setiap itemset
print("Itemset Support:")
for itemset, support in itemset_support.items():
    print(f"{set(itemset)}: {support}")

# Generate all subsets of a given itemset
def subsets(itemset):
    return chain(*[combinations(itemset, i + 1) for i in range(len(itemset))])

# Menghitung association rules
association_rules = []
for itemset in frequent_itemsets:
    for subset in subsets(itemset):
        subset = frozenset(subset)
        remaining = frozenset(itemset) - subset
        if len(remaining) > 0:
            confidence = itemset_support[frozenset(itemset)] / itemset_support[subset]
            lift = confidence / (itemset_support[remaining] / len(transactions))
            rule = (subset, remaining, confidence, lift)
            association_rules.append(rule)

# Menampilkan association rules
print("Association Rules:")
for rule in association_rules:
    print(f"{set(rule[0])} -> {set(rule[1])} (confidence: {rule[2]:.2f}, lift: {rule[3]:.2f})")

Itemset Support:
{'Roti'}: 4
{'Susu'}: 4
{'Susu', 'Roti'}: 3
{'Susu', 'Mentega'}: 3
{'Mentega'}: 4
{'Mentega', 'Roti'}: 3
Association Rules:
{'Susu'} -> {'Roti'} (confidence: 0.75, lift: 0.94)
{'Roti'} -> {'Susu'} (confidence: 0.75, lift: 0.94)
{'Susu'} -> {'Mentega'} (confidence: 0.75, lift: 0.94)
{'Mentega'} -> {'Susu'} (confidence: 0.75, lift: 0.94)
{'Mentega'} -> {'Roti'} (confidence: 0.75, lift: 0.94)
{'Roti'} -> {'Mentega'} (confidence: 0.75, lift: 0.94)
